In [28]:
import os
import sys
import yaml
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *
import requests
from ibm_ai_openscale.utils import get_instance_guid
import ibm_watson_machine_learning
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
import ibm_watson_openscale


with open("../credentials.yaml") as stream:
    try:
        credentials = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


with open("../metadata.yaml") as stream:
    try:
        metadata = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


service_credentials = {
    "apikey": credentials["apikey"],
    "url": "https://api.aiopenscale.cloud.ibm.com",
}

DEPLOYMENT_UID = metadata["deployment_uid"]
MODEL_UID = metadata["model_uid"]
MODEL_NAME = metadata["project_name"] + "_" + metadata["project_version"]
SPACE_ID = credentials["space_id"]
WOS_GUID = get_instance_guid(api_key=service_credentials["apikey"])
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": service_credentials["apikey"],
    "url": "https://api.aiopenscale.cloud.ibm.com",
}

if WOS_GUID is None:
    print("Watson OpenScale GUID NOT FOUND")
else:
    print(WOS_GUID)
    
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
print(ai_client.version)

wml_credentials = {"url": credentials["url"], "apikey": credentials["apikey"]}

wml_client = ibm_watson_machine_learning.APIClient(wml_credentials)

wml_credentials = {
    "url": credentials["url"],
    "apikey": credentials["apikey"],
    "instance_id": "wml_local",
}

wml_client.set.default_space("16148a4d-9055-4220-af26-0c0369cdf31a")

authenticator = IAMAuthenticator(apikey=credentials["apikey"])
wos_client = ibm_watson_openscale.APIClient(
    authenticator=authenticator, 
    service_url="https://api.aiopenscale.cloud.ibm.com")




7b220d7b-3196-43c3-8510-a4f710021a04
2.2.1
DEPRECATED!! Python 3.6 framework is deprecated and will be removed on Jan 20th, 2021. It will be read-only mode starting Nov 20th, 2020. i.e you won't be able to create new assets using this client. Use Python 3.7 instead. For details, see https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


'SUCCESS'

In [292]:

KEEP_MY_INTERNAL_POSTGRES = True
DB_CREDENTIALS = None
try:
    data_mart_details = ai_client.data_mart.get_details()
    if (
        "internal_database" in data_mart_details
        and data_mart_details["internal_database"]
    ):
        if KEEP_MY_INTERNAL_POSTGRES:
            print("Using existing internal datamart.")
        else:
            if DB_CREDENTIALS is None:
                print(
                    "No postgres credentials supplied. Using existing internal datamart"
                )
            else:
                print("Switching to external datamart")
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print("Using existing external datamart")
except:
    if DB_CREDENTIALS is None:
        print("Setting up internal datamart")
        ai_client.data_mart.setup(internal_db=True)
    else:
        print("Setting up external datamart")
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print("Setup failed, trying Db2 setup")
            ai_client.data_mart.setup(
                db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS["username"]
            )
data_mart_details = ai_client.data_mart.get_details()

binding_uid = ai_client.data_mart.bindings.add(
    "Rain Aus", WatsonMachineLearningInstance(wml_credentials)
)

bindings_details = ai_client.data_mart.bindings.get_details()

if binding_uid is None:
    binding_uid = [
        binding["metadata"]["guid"]
        for binding in bindings_details["service_bindings"]
        if binding["entity"]["name"] == "WML Cloud Instance"
    ][0]
ai_client.data_mart.bindings.list()

ai_client.data_mart.bindings.list_assets(binding_uid=binding_uid)

Using existing internal datamart.
Operation 'engine binding' is supported for Watson Machine Learning in Dallas or Frankfurt regions
Warning during bind instance.
Status code: 409, body: {"errors":[{"code":"AIQCS0010W","message":"Service Binding with this id is already defined"}],"trace":"NWRlMDQ0NmQtNDRkZS00MjcwLWIyYWEtYTAwYjk2Y2M5N2Vm"}


wml_local,Rain Australia,watson_machine_learning,2021-04-04T21:16:49.550Z
974615e0-4c06-40a3-af1f-944192f7e472,Rain Aus,watson_machine_learning,2021-04-01T18:34:08.733Z


Status code: 500, body: {"trace":"java.lang.RuntimeException: AIQGS0023E : BindingOrInstanceId = wml_local, ServiceType = watson_machine_learning, SubscriptionId = null : GetDeployments failed org.apache.http.client.ClientProtocolException: AIQGS0167E : HTTP call failed (no of calls attempted = 1, method = GET, url = https://us-south.ml.cloud.ibm.com/v3/wml_instances/wml_local/deployments, status = 403)..\n\tat com.ibm.aios.mlgateway.discovery.impl.WMLDeployments.getDeploymentCollectionBySpaceId(WMLDeployments.java:399)\n\tat com.ibm.aios.mlgateway.discovery.impl.WMLDeployments.getDeploymentCollection(WMLDeployments.java:247)\n\tat com.ibm.aios.mlgateway.discovery.impl.WMLDeployments.getDeployments(WMLDeployments.java:224)\n\tat com.ibm.aios.mlgateway.api.server.impl.DeploymentCollectionFactory.getDeploymentCollection(DeploymentCollectionFactory.java:102)\n\tat com.ibm.aios.mlgateway.api.server.impl.V1ApiServiceImpl.getDeploymentsByBindingId(V1ApiServiceImpl.java:185)\n\tat com.ibm.aio

In [30]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
# for subscription in subscriptions_uids:
#     sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
#     if sub_name == MODEL_NAME:
#         ai_client.data_mart.subscriptions.delete(subscription)
#         print('Deleted existing subscription for', MODEL_NAME)

In [293]:
# subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
#     MODEL_UID,
#     problem_type=ProblemType.BINARY_CLASSIFICATION,
#     input_data_type=InputDataType.STRUCTURED,
#     label_column='RainTomorrow',
#     prediction_column='predictedLabel',
#     probability_column='probability',
#     transaction_id_column='transaction_id',
#     feature_columns = ["Humidity3pm", "Humidity9am", "MaxTemp", "MinTemp", "Pressure3pm", "Pressure9am", "RainToday", "Rainfall", "Temp3pm", "Temp9am", "WindDir3pm_E", "WindDir3pm_ENE", "WindDir3pm_ESE", "WindDir3pm_N", "WindDir3pm_NE", "WindDir3pm_NNE", "WindDir3pm_NNW", "WindDir3pm_NW", "WindDir3pm_S", "WindDir3pm_SE", "WindDir3pm_SSE", "WindDir3pm_SSW", "WindDir3pm_SW", "WindDir3pm_W", "WindDir3pm_WNW", "WindDir3pm_WSW", "WindDir9am_E", "WindDir9am_ENE", "WindDir9am_ESE", "WindDir9am_N", "WindDir9am_NE", "WindDir9am_NNE", "WindDir9am_NNW", "WindDir9am_NW", "WindDir9am_S", "WindDir9am_SE", "WindDir9am_SSE", "WindDir9am_SSW", "WindDir9am_SW", "WindDir9am_W", "WindDir9am_WNW", "WindDir9am_WSW", "WindGustDir_E", "WindGustDir_ENE", "WindGustDir_ESE", "WindGustDir_N", "WindGustDir_NE", "WindGustDir_NNE", "WindGustDir_NNW", "WindGustDir_NW", "WindGustDir_S", "WindGustDir_SE", "WindGustDir_SSE", "WindGustDir_SSW", "WindGustDir_SW", "WindGustDir_W", "WindGustDir_WNW", "WindGustDir_WSW", "WindGustSpeed", "WindSpeed3pm", "WindSpeed9am"],
#     categorical_columns = ["RainToday", "WindDir3pm_E", "WindDir3pm_ENE", "WindDir3pm_ESE", "WindDir3pm_N", "WindDir3pm_NE", "WindDir3pm_NNE", "WindDir3pm_NNW", "WindDir3pm_NW", "WindDir3pm_S", "WindDir3pm_SE", "WindDir3pm_SSE", "WindDir3pm_SSW", "WindDir3pm_SW", "WindDir3pm_W", "WindDir3pm_WNW", "WindDir3pm_WSW", "WindDir9am_E", "WindDir9am_ENE", "WindDir9am_ESE", "WindDir9am_N", "WindDir9am_NE", "WindDir9am_NNE", "WindDir9am_NNW", "WindDir9am_NW", "WindDir9am_S", "WindDir9am_SE", "WindDir9am_SSE", "WindDir9am_SSW", "WindDir9am_SW", "WindDir9am_W", "WindDir9am_WNW", "WindDir9am_WSW", "WindGustDir_E", "WindGustDir_ENE", "WindGustDir_ESE", "WindGustDir_N", "WindGustDir_NE", "WindGustDir_NNE", "WindGustDir_NNW", "WindGustDir_NW", "WindGustDir_S", "WindGustDir_SE", "WindGustDir_SSE", "WindGustDir_SSW", "WindGustDir_SW", "WindGustDir_W", "WindGustDir_WNW", "WindGustDir_WSW", "WindGustSpeed", "WindSpeed3pm", "WindSpeed9am"]
# ))

subscription = None

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()

    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Subscription already exists; get the existing one


In [294]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,Rain_aus_v0.3,model,974615e0-4c06-40a3-af1f-944192f7e472,2021-04-04T21:23:38.668Z


In [295]:

for deployment in wml_client.deployments.get_details()['resources']:
    if DEPLOYMENT_UID in deployment['metadata']['id']:

        scoring_endpoint = deployment['entity']['status']['online_url']['url']
        
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/e02e481d-4e56-470f-baa9-ae84a583c0a8/predictions


In [282]:
data = pd.read_csv("../data/weatherAUS_processed.csv")

X = data.iloc[:, :-1]
y = data[data.columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.01, random_state=1337
)

payload_scoring = {
    "input_data": [
        {
            "fields": X.columns.to_numpy().tolist(),
            "values": X_test.to_numpy().tolist(),
        }
    ]
}

In [252]:
scoring_response = wml_client.deployments.score(DEPLOYMENT_UID, payload_scoring)

# scoring_response

In [253]:
pred_values = np.squeeze(scoring_response["predictions"][0]["values"])
y_pred_values = [i[0] for i in pred_values]

def comb_eval(y, y_pred):
    acc = accuracy_score(y, y_pred)
    recall = recall_score(y, y_pred)
    precision = precision_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    return {"accuracy": acc, "recall": recall, "precision": precision, "f1": f1}

eval = comb_eval(y_test, y_pred_values)
print(eval)


{'accuracy': 0.8704318936877077, 'recall': 0.9724284199363733, 'precision': 0.8758357211079274, 'f1': 0.9216080402010052}


In [255]:
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
print("Payload data set id:", payload_data_set_id)




Payload data set id: f4791725-24f8-4a00-9c13-b331ebca47f6


In [256]:
print('Logging')
records = [PayloadRecord(request=payload_scoring, response=scoring_response, response_time=72)]
store_record_info = wos_client.data_sets.store_records(payload_data_set_id,records)


Logging


In [283]:
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result

feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
    sys.exit(1)

data = X_test.to_dict('records')

wos_client.data_sets.store_records(
    feedback_dataset_id, 
    request_body=data, 
    background_mode=False,
    header=True,
    delimiter=',',
    csv_max_line_length=1000)

print(wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id))




 Waiting for end of storing records with request id: 157c3777-25dd-4463-8920-45a32d164f93 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


1325


In [296]:

from ibm_watson_openscale.supporting_classes.enums import *

print('\nData marts: ')
datams = wos_client.data_marts.list().result.data_marts
for d in datams:
    print(d.metadata.id)
datamart_id = d.metadata.id

print('\nService providers: ')
services = wos_client.service_providers.list().result.service_providers
for service in services:
    print(service.metadata.id+" / Name: "+service.entity.name)
service_id = service.metadata.id

#wos_client.subscriptions.show()
#wos_client.data_sets.show()

print('\nSubscriptions: ')
subscriptions = wos_client.subscriptions.list(data_mart_id=datamart_id, service_provider_id=service_id).result.subscriptions
for s in subscriptions:
  print(s.metadata.id+"   "+s.entity.asset.name)
subscription_id = s.metadata.id

print('\n')

payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
print("Payload data set id:", payload_data_set_id)


pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")



Data marts: 
7b220d7b-3196-43c3-8510-a4f710021a04

Service providers: 
wml_local / Name: Rain Australia
974615e0-4c06-40a3-af1f-944192f7e472 / Name: Rain Aus

Subscriptions: 
bb7a45c3-15ad-4932-aeb8-8d32d54b8b05   Rain_aus_v0.3


Payload data set id: f4791725-24f8-4a00-9c13-b331ebca47f6
Number of records in the payload logging table: 4460


In [278]:
target = ibm_watson_openscale.base_classes.watson_open_scale_v2.Target(
            target_type=TargetTypes.SUBSCRIPTION,
            target_id=subscription.uid
        )
parameters =  {
            "min_feedback_data_size": 200
        }
thresholds =  [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.75
    }]
wos_client.monitor_instances.create(
        data_mart_id=datamart_id,
        background_mode=False,
        monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
        target=target,
        parameters=parameters,
        thresholds=thresholds
)

ApiException: Error: Resource could not be created because of conflict with existing resource `5ddff093-25fa-44f8-abae-fd29659fd0d0` of type `monitor_instance`, Code: 409 , X-global-transaction-id: NzE3YTdlNDUtMzlmYy00NGUxLWE5YWUtOTZhMmViMmUxYzk0

In [277]:
wos_client.data_sets.print_records_schema(data_set_id=payload_data_set_id)

In [134]:
# records = [PayloadRecord(request=payload_scoring, response=scoring_response, response_time=72)]


x = wos_client.data_sets.show_records(data_set_id=payload_data_set_id)
x

0.0,,0.0,0.0,52.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-1,0.0,0.0,0.0,95.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.6449555088313681,0.0,1019.9,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,0,0.0,81.0,0.0,0.0,None,None,1014.7,0.0,0.0,1.0,24.0,0.0,1.0,10.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.2,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,2.7,0.0,0.0,None,0.0,0.0,0.0,0.0,28.0,0.0,0.0,11.2,0.0,"[0.6449555088313681, 0.3550444911686318]",0.0,1.0
0.0,,0.0,1.0,61.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-10,0.0,0.0,0.0,56.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.8391123182628472,0.0,1013.6,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,1,0.0,61.0,0.0,0.0,None,None,1014.1,0.0,0.0,0.0,28.0,0.0,1.0,23.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,21.6,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,16.8,0.0,0.0,None,0.0,0.0,0.0,0.0,39.0,0.0,0.0,24.6,1.0,"[0.1608876817371521, 0.8391123182628472]",0.0,0.0
0.0,,0.0,0.0,41.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-100,0.0,0.0,0.0,66.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.8846832430208341,0.0,1020.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,1,0.0,55.0,0.0,0.0,None,None,1017.3,0.0,0.0,1.0,19.0,0.0,1.0,22.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.7,e02e481d-4e56-470f-baa9-ae84a583c0a8,1.0,14.2,0.0,0.0,None,0.0,1.0,0.0,0.0,31.0,0.0,0.0,24.2,0.0,"[0.11531675697916585, 0.8846832430208341]",0.0,0.0
0.0,,0.0,0.0,46.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-101,0.0,0.0,0.0,60.0,0.0,None,2021-04-05T15:20:14.621222Z,1.0,0.0,0.0,0.0,0.0,None,0.8809851924975399,0.0,1014.9,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,1,0.0,32.0,0.0,1.0,None,None,1016.4,0.0,0.0,0.0,22.0,0.0,1.0,19.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.9,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,12.4,0.0,0.0,None,0.0,0.0,0.0,0.0,28.0,0.0,0.0,27.0,0.0,"[0.11901480750246042, 0.8809851924975399]",1.0,0.0
0.0,,0.0,0.0,33.0,None,0.6,4a84d1b0d13e795d735345f6cd18a90c-102,0.0,0.0,0.0,30.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.8491193551880492,0.0,1016.2,0.0,0.0,0.0,None,0.0,0.0,0.0,1.0,1,0.0,63.0,0.0,0.0,None,None,1012.4,0.0,0.0,1.0,20.0,0.0,1.0,31.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,18.3,0.0,0.0,None,0.0,0.0,0.0,0.0,17.0,0.0,0.0,32.9,0.0,"[0.15088064481195082, 0.8491193551880492]",0.0,0.0
0.0,,0.0,0.0,33.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-103,0.0,0.0,0.0,49.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.8543884658319435,0.0,1004.4,0.0,0.0,0.0,None,0.0,1.0,0.0,0.0,1,0.0,58.0,0.0,0.0,None,None,1002.2,0.0,0.0,0.0,7.0,0.0,1.0,35.4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32.6,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,27.3,0.0,0.0,None,0.0,0.0,0.0,0.0,15.0,0.0,0.0,37.1,0.0,"[0.14561153416805678, 0.8543884658319435]",0.0,0.0
1.0,,0.0,0.0,35.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-104,0.0,0.0,0.0,44.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.9484440672707255,0.0,1019.8,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,1,0.0,42.0,0.0,0.0,None,None,1016.0,0.0,0.0,0.0,15.0,0.0,1.0,27.9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,24.5,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,15.4,0.0,0.0,None,0.0,0.0,0.0,1.0,26.0,0.0,0.0,29.8,0.0,"[0.05155593272927421, 0.9484440672707255]",0.0,0.0
0.0,,0.0,0.0,28.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-105,0.0,0.0,0.0,34.0,0.0,None,2021-04-05T15:20:14.621222Z,1.0,0.0,0.0,0.0,0.0,None,0.926856429086007,0.0,1019.2,0.0,0.0,0.0,None,0.0,0.0,1.0,0.0,1,0.0,75.0,0.0,0.0,None,None,1015.5,0.0,0.0,0.0,13.0,0.0,1.0,25.8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.5,e02e481d-4e56-470f-baa9-ae84a583c0a8,0.0,11.7,0.0,0.0,None,0.0,0.0,0.0,0.0,13.0,0.0,0.0,27.1,0.0,"[0.07314357091399265, 0.926856429086007]",0.0,0.0
0.0,,0.0,1.0,17.0,None,0.0,4a84d1b0d13e795d735345f6cd18a90c-106,0.0,0.0,0.0,51.0,0.0,None,2021-04-05T15:20:14.621222Z,0.0,0.0,0.0,0.0,0.0,None,0.9265235513246206,0.0,1026.0,0.0,1.0,0.0,None,1.0,0.0,0.0,0.0,1,0.0,71.0,0.0,0.0,None,None,1023.1,0.0,0.0,0.0,6.0,0.0,1.0,22.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.2,e0

In [184]:
records2 = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id,
                                                   output_type=ResponseTypes.PANDAS
   )
df = records2.result

import pandas as pd
import numpy as np
# df.to_csv('res.csv',index=False)
df['RainTomorrow'] = np.random.randint(2, size=df.shape[0])#.toarray().tolist()
df.to_csv('res.csv',index=False)



In [299]:
wos_client.data_sets.get_records_count(payload_data_set_id) # ibm_watson_openscale


4460

In [248]:
monitor_instances_info = wos_client.monitor_instances.show(
    data_mart_id="7b220d7b-3196-43c3-8510-a4f710021a04",
     )


7b220d7b-3196-43c3-8510-a4f710021a04,active,bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,subscription,quality,2021-04-08 16:25:48.434000+00:00,5ddff093-25fa-44f8-abae-fd29659fd0d0
7b220d7b-3196-43c3-8510-a4f710021a04,active,bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,subscription,drift,2021-04-05 15:16:56.829000+00:00,9d51655d-1f57-4061-9df4-8ef20b6ebced
7b220d7b-3196-43c3-8510-a4f710021a04,error,bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,subscription,fairness,2021-04-04 21:30:28.742000+00:00,30bc90c4-8c54-413d-98aa-8d4ff5bb2680
7b220d7b-3196-43c3-8510-a4f710021a04,error,bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,subscription,explainability,2021-04-04 21:28:01.896000+00:00,dcb0abd5-b476-43c0-ba7f-be93ad250d6f
7b220d7b-3196-43c3-8510-a4f710021a04,active,bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,subscription,mrm,2021-04-04 21:23:42.492000+00:00,3748dfc5-7236-406b-be1d-1ca2363c9343
7b220d7b-3196-43c3-8510-a4f710021a04,active,bb7a45c3-15ad-4932-aeb8-8d32d54b8b05,subscription,performance,2021-04-04 21:23:42.488000+00:00,570ab6f2-e0e3-4402-bb78-d8de41331a3e
7b220d7b-3196-43c3-8510-a4f710021a04,active,6812f06b-1a26-4eba-93cf-91b520950a74,instance,performance,2021-03-10 18:28:38.861000+00:00,26bb6bb7-d140-4280-8044-63ec11f7af3b


In [246]:
wos_client.monitor_instances.show_metrics(monitor_instance_id='5ddff093-25fa-44f8-abae-fd29659fd0d0')

In [232]:
# wos_client.monitor_instances.delete(
#         background_mode=False,
#         monitor_instance_id='94e582d5-c244-4533-9697-c16046c5fc40'
#      )





 Waiting for end of deleting monitor instance 94e582d5-c244-4533-9697-c16046c5fc40 




finished

-------------------------------------------------
 Successfully finished deleting monitor instance 
-------------------------------------------------




In [300]:
monitor_instance_run_info = wos_client.monitor_instances.run(
        background_mode=False,
        monitor_instance_id='5ddff093-25fa-44f8-abae-fd29659fd0d0'
     )




 Waiting for end of monitoring run 2187b9be-1ceb-47ad-ae2b-1e83528c3b3e 




error

-------------------------------
 Run failed with status: error 
-------------------------------


Reason: ['code: AIQFS0002E, message: Action `Score Batch` has failed with status code 400; associated message: `{2}`', 'code: AIQGS0099E, message: X has 66 features, but this StandardScaler is expecting 61 features as input.']
